In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder

import pickle as pk

jtplot.style(figsize=(15, 9))

In [2]:
map_data = pd.read_csv('../data/SHR65_22.csv', index_col=0)
map_data.head()

,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,Incident,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
ID,,,,,,,,,,,,,,,,,,,,,
197601001AKASP00,"Juneau, AK",AKASP00,Alaska,State Troopers,Primary state LE,FBI,Yes,1976,January,1,...,American Indian or Alaskan Native,Unknown or not reported,Knife or cutting instrument,Husband,Other arguments,NaN,0,0,30180.0,Rural Alaska
197601001AL00102,"Jefferson, AL",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00104,"Jefferson, AL",AL00104,Alabama,Fairfield,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00106,"Jefferson, AL",AL00106,Alabama,Leeds,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00201,"Mobile, AL",AL00201,Alabama,Mobile,Municipal police,FBI,Yes,1976,January,1,...,White,Unknown or not reported,Strangulation - hanging,Acquaintance,Circumstances undetermined,NaN,0,0,30180.0,"Mobile, AL"


In [3]:
useless_clmns = ['FileDate', 'Source', 'ActionType', 'Ori', 'Subcircum'] # 'Solved'
map_data.drop(useless_clmns, axis=1, inplace=True)

In [4]:
trans_clmns = map_data.dtypes[map_data.dtypes != 'int64'].index.to_list()

In [5]:
yes = map_data[(map_data['Solved']=='Yes')
               & ~(map_data['OffSex']=='Unknown')
               & ~(map_data['OffRace']=='Unknown')
              ].index
no = map_data[map_data['Solved']=='No'].index # test

In [6]:
yes.shape, no.shape

((602887,), (256868,))

In [7]:
# Relationship not determined / Circumstances undetermined

In [8]:
enc = OrdinalEncoder()
enc.fit(map_data[trans_clmns])
map_data[trans_clmns] = enc.transform(map_data[trans_clmns])

In [9]:
yes = map_data.loc[yes]
no = map_data.loc[no]

In [10]:
map_data

,CNTYFIPS,State,Agency,Agentype,Solved,Year,Month,Incident,Homicide,Situation,...,OffAge,OffSex,OffRace,OffEthnic,Weapon,Relationship,Circumstance,VicCount,OffCount,MSA
ID,,,,,,,,,,,,,,,,,,,,,
197601001AKASP00,1426.0,1.0,8431.0,3.0,1.0,1976,4.0,1,1.0,4.0,...,55,0.0,0.0,2.0,7.0,14.0,24.0,0,0,282.0
197601001AL00102,1370.0,0.0,666.0,1.0,1.0,1976,4.0,1,1.0,4.0,...,67,1.0,2.0,2.0,15.0,0.0,12.0,0,0,35.0
197601001AL00104,1370.0,0.0,2632.0,1.0,1.0,1976,4.0,1,1.0,4.0,...,53,1.0,2.0,2.0,15.0,28.0,22.0,0,0,35.0
197601001AL00106,1370.0,0.0,4492.0,1.0,1.0,1976,4.0,1,1.0,4.0,...,35,0.0,2.0,2.0,7.0,2.0,24.0,0,0,35.0
197601001AL00201,1895.0,0.0,5254.0,1.0,1.0,1976,4.0,1,1.0,4.0,...,25,0.0,5.0,2.0,16.0,0.0,10.0,0,0,216.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202208001WY01300,1995.0,50.0,5536.0,5.0,1.0,2022,1.0,1,1.0,1.0,...,26,1.0,5.0,1.0,6.0,11.0,21.0,1,0,54.0
202211001WY01701,2540.0,50.0,7356.0,1.0,1.0,2022,9.0,1,0.0,4.0,...,34,1.0,5.0,1.0,8.0,0.0,1.0,0,0,328.0
202206001WY01901,2682.0,50.0,3224.0,1.0,1.0,2022,6.0,1,1.0,1.0,...,51,1.0,5.0,1.0,6.0,18.0,22.0,1,0,328.0


In [11]:
X = yes.drop('OffRace', axis=1)
y = yes['OffRace']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
_model = RandomForestClassifier(max_depth=4).fit(X_train, y_train)
_model.score(X_test, y_test)

0.8589095991719818

In [13]:
prediction = _model.predict(no.drop('OffRace', axis=1))
no['OffRace'] = prediction
no[trans_clmns] = enc.inverse_transform(no[trans_clmns])
no.head()

,CNTYFIPS,State,Agency,Agentype,Solved,Year,Month,Incident,Homicide,Situation,...,OffAge,OffSex,OffRace,OffEthnic,Weapon,Relationship,Circumstance,VicCount,OffCount,MSA
ID,,,,,,,,,,,,,,,,,,,,,
197601001AR06002,"Pulaski, AR",Arkansas,Little Rock,Municipal police,No,1976,January,1,Murder and non-negligent manslaughter,Single victim/unknown offender(s),...,999,Unknown,White,Unknown or not reported,"Handgun - pistol, revolver, etc",Relationship not determined,Circumstances undetermined,0,0,"Little Rock-North Little Rock, AR"
197601001AZ00700,"Maricopa, AZ",Arizona,Maricopa County,Sheriff,No,1976,January,1,Murder and non-negligent manslaughter,Single victim/unknown offender(s),...,999,Unknown,White,Unknown or not reported,Knife or cutting instrument,Relationship not determined,Circumstances undetermined,0,0,"Phoenix-Mesa-Scottsdale, AZ"
197601001AZ01307,"Yavapai, AZ",Arizona,Prescott,Municipal police,No,1976,January,1,Murder and non-negligent manslaughter,Single victim/unknown offender(s),...,999,Unknown,White,Unknown or not reported,Fire,Relationship not determined,Arson,0,0,"Prescott, AZ"
197601001CA00701,"Contra Costa, CA",California,Antioch,Municipal police,No,1976,January,1,Murder and non-negligent manslaughter,Single victim/single offender,...,24,Unknown,White,Unknown or not reported,"Personal weapons, includes beating",Daughter,Other,0,0,"San Francisco-Oakland-Fremont, CA"
197601001CA01200,"Humboldt, CA",California,Humboldt County,Sheriff,No,1976,January,1,Murder and non-negligent manslaughter,Single victim/unknown offender(s),...,999,Unknown,White,Unknown or not reported,"Personal weapons, includes beating",Relationship not determined,Rape,0,0,Rural California


In [14]:
with open('ml/offrace_RF.pk', 'wb') as f:
    pk.dump(_model, f, protocol=5)